In [1]:
import json
import torch
import torch.nn as nn
import random
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [3]:
dev_dict = json.load(open('../data/coqa-dev-v1.0.json', encoding='utf8'))

In [4]:
train_list = json.load(open('../data/qa_train_list.json', encoding='utf8'))
dev_list = json.load(open('../data/qa_dev_list.json', encoding='utf8'))

In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def batchify(data, n):
    len_dict = {}
    for item in data:
        length = item.shape[1]
        try:
            len_dict[length].append(item)
        except:
            len_dict[length] = [item]

    batch_chunks = []
    for k in len_dict.keys():
        vectors = len_dict[k]
        batch_chunks += chunks(vectors, n)

    batches = []
    for chunk in batch_chunks:
        inputs = torch.stack([item[0] for item in chunk])
        batches.append((inputs))

    return batches

In [6]:
_limit = 1024
data = []
total_trimmed = 0
for item in train_list:
    tokens = tokenizer.encode(item, return_tensors='pt')
    if tokens.shape[1] > _limit:
        tokens = tokens[:, :_limit]
        total_trimmed += 1
    data.append(tokens)
print(f'Trimmed {total_trimmed} out of {len(train_list)}')

Token indices sequence length is longer than the specified maximum sequence length for this model (1088 > 1024). Running this sequence through the model will result in indexing errors


Trimmed 52 out of 7199


In [6]:
train_batches = batchify(data, 1)

NameError: name 'data' is not defined

In [5]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [8]:
def train(train_model, batches, optimizer, criterion):
    total_loss = 0.
    for i, batch in tqdm(enumerate(batches), total=len(batches)):
        model.train()
        inputs = batch
        optimizer.zero_grad()
        loss = train_model(inputs.cuda(), labels=inputs.cuda())[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(train_model.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(batches)

def test(test_model, batches):
    total_loss = 0.
    for i, batch in tqdm(enumerate(batches), total=len(batches)):
        test_model.eval()
        inputs = batch
        loss = test_model(inputs, labels=inputs)[0]
        total_loss += loss.item()

    return total_loss / len(batches)

In [9]:
from torch.optim.lr_scheduler import StepLR

random.shuffle(train_batches)
scheduler = StepLR(optimizer, step_size=2, gamma=0.8)
for epoch in range(10):
    random.shuffle(train_batches)
    loss = train(model, train_batches, optimizer, criterion)
    print('Epoch:', epoch, 'Loss:', loss)
    torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict()},
                'save_small' + str(epoch))
    scheduler.step()

NameError: name 'train_batches' is not defined

In [6]:
_question_prompt = '\nQ: '
_answer_prompt = '\nA: '
    
def get_text_up_to_question_number(text, number):
    pos = text.find(_answer_prompt)
    for _ in range(number):
        pos = text.find(_answer_prompt, pos + 1)
    return text[0:pos + 1]
    
def get_answers_number(text):
    return text.count(_answer_prompt)

def get_answer_number(text, number):
    pos = text.find(_answer_prompt)
    for _ in range(number):
        pos = text.find(_answer_prompt, pos + 1)
    end = text.find('\n', pos + len(_answer_prompt))
    return text[pos + len(_answer_prompt):end]

def get_question_number(text, number):
    pos = text.find(_question_prompt)
    for _ in range(number):
        pos = text.find(_question_prompt, pos + 1)
    end = text.find('\n', pos + len(_question_prompt))
    return text[pos + len(_question_prompt):end]

def get_all_answers(dev_dict, dev_index):
    answers = [[item['input_text'] for item in dev_dict['data'][dev_index]['answers']]]
    answers += [[item['input_text'] for item in dev_dict['data'][dev_index]['additional_answers'][str(index)]] for index in range(3)]
    return [list(set([answers[j][i] for j in range(len(answers))])) for i in range(len(answers[0]))]

In [7]:
def generate_answer_number(model, text, number):
    prompt = get_text_up_to_question_number(text, number)
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 20
    tokens_length = tokens.shape[1]
    output = model.generate(
             tokens,
             max_length=tokens_length + _length,
             temperature=0,
             pad_token_id=50256
    )
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return get_answer_number(output, number)

In [8]:
def compute_accuracy_of_model_with_all_answers(model):
    total_number_of_questions = 0
    correct_answers = 0
    wrong_predictions = []

    dlist = dev_list[:10]
    for index, text in tqdm(enumerate(dlist), total=len(dlist)):
        total_questions = get_answers_number(text)
        all_answers = get_all_answers(dev_dict, index)
        for number in range(total_questions):
            prediction = generate_answer_number(model, text, number)
            prediction = prediction.replace('.', '').replace('"', '')
            it_was_answered = False
            for label in all_answers[number]:
                label = label.replace('.', '').replace('"', '')
                if prediction.lower() == label.lower():
                    correct_answers += 1
                    it_was_answered = True
                    break
                elif prediction.lower() in label.lower():
                    correct_answers += 1
                    it_was_answered = True
                    break
                elif label.lower() in prediction.lower():
                    correct_answers += 1
                    it_was_answered = True
                    break
                else:
                    wrong_predictions.append({'label': label, 'prediction': prediction})
            if not it_was_answered:
                print('No Answer for number', number)
            total_number_of_questions += 1

    return correct_answers/total_number_of_questions, wrong_predictions

In [9]:
def get_text_from_data_item(item, max_num_questions=0, question_number=-1, last_question=True):
    text = 'In the text below two people are discussing a story.\n\n'
    text += 'Story:\n' + item['story'] + '\n\n'
    text += 'Discussion:\n'
    text += '\n'.join(['Q: ' + q['input_text'] 
                       + '\nA: ' + a['input_text'] 
                       for q, a in zip(item['questions'][max(0,question_number-max_num_questions):question_number+1], 
                                       item['answers'][max(0,question_number-max_num_questions):question_number+1]) 
                      ])
    if not last_question:
        text = '\n'.join(text.split('\n')[:-1]) + '\n'
    return text

In [10]:
def generate_answer(model, prompt):
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 50
    tokens_length = tokens.shape[1]
    if tokens_length + _length > 1024:
        return ''
    output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length,
             #temperature=0,
             pad_token_id=50256
    )
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    offset = len(prompt)
    start = offset + 1
    end = output.find('\n', start)
    return output[start:end].split(':')[-1].strip()

In [11]:
def generate_multiple_answers(model, prompt, num_replicas=25):
    model.train()
    outputs = []
    with torch.no_grad():
        tokens = tokenizer.encode(prompt, return_tensors='pt')
        tokens = tokens.repeat(num_replicas,1)
        _length = 50
        tokens_length = tokens.shape[1]
        if tokens_length + _length > 1024:
            return ''

        
        output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length,
             pad_token_id=50256
        )
        for index in range(num_replicas):
            text = tokenizer.decode(output[index, :], skip_special_tokens=True)
            offset = len(prompt)
            start = offset + 1
            end = text.find('\n', start)
            outputs.append(text[start:end].split(':')[-1].strip())

    return outputs

In [12]:
device = 'cuda'

In [13]:
import numpy as np
from sentence_transformers import SentenceTransformer


sentence_model = SentenceTransformer('msmarco-distilbert-base-v3')
sentence_model = sentence_model.to(device)

In [14]:
def get_embeddings_from_text(text):
    outputs = sentence_model.encode(text)
    return outputs

def group_similar_answers_and_get_scores(answers):
    answers_dict = {}
    threshold = 0.7
    embeddings = get_embeddings_from_text(answers)
    embeddings = np.array([e/np.linalg.norm(e) for e in embeddings])
    similarity_matrix = np.matmul(embeddings, embeddings.transpose())
    superseded = set()
    superseded_from = {}
    for i in range(len(answers)):
        for j in range(len(answers)):
            if i > j:
                continue
            if i != j and answers[i] == answers[j]:
                continue
            if similarity_matrix[i][j] > threshold :
                answers_dict.setdefault(i, 0)
                answers_dict[i] += 1
                if i != j:
                    superseded.add(j)
                    superseded_from.setdefault(i, [])
                    superseded_from[i].append(j)

    answers_and_scores = [(index, score/len(answers))
                          for index, score in answers_dict.items() 
                          if index not in superseded]
    
    new_scores_dict = {}
    total_score = sum(item[1] for item in answers_and_scores)
    for answer_index, score in answers_and_scores:
        answer_group = [answers[answer_index]]
        if answer_index in superseded_from:
            answer_group += [answers[i] for i in superseded_from[answer_index]]
        answer_group = tuple(set(answer_group))
        if answer_group in new_scores_dict:
            new_scores_dict[answer_group] += score / total_score
        else:
            new_scores_dict[answer_group] = score / total_score
    
    
    return sorted(list(new_scores_dict.items()), key=lambda x: -x[1])

In [15]:
group_similar_answers_and_get_scores(['white', 'white', 'white', 'white', 'orange', 'orange', 'white', 'white', 'white', 'white', 'white', 'orange', 'white', 'white', 'White', 'white', 'orange', 'white', 'white', 'white']
)

[(('white', 'White'), 0.846153846153846), (('orange',), 0.15384615384615385)]

In [16]:
small_text = get_text_from_data_item(dev_dict['data'][0], 
                                     max_num_questions=8,
                                     question_number=1,
                                     last_question=False)

answers = generate_multiple_answers(model, small_text)
group_similar_answers_and_get_scores(answers)

[(('in a barn',
   'in the barn',
   'In the barn',
   'in a barn near a farm house.',
   'In the barn.'),
  0.2727272727272727),
 (('',), 0.22727272727272724),
 (('What is Cotton?', 'Cotton'), 0.0909090909090909),
 (('She lived in a barn in a barn.',
   'She lived in a barn in a farm near a farm. farm.'),
  0.0909090909090909),
 (('The farm house in the middle of the woods.',), 0.04545454545454545),
 (("Cotton's mommy",), 0.04545454545454545),
 (('Her mommy and 5 sisters were all white.',), 0.04545454545454545),
 (('Where did she live?',), 0.04545454545454545),
 (('Cotton lived in a nice, warm place above the barn.',),
  0.04545454545454545),
 (('"I was born in a barn in a barn in a barn in a barn in a barn in a barn in a barn in a barn in a barn in a barn in a barn in a barn in a barn in a barn i',),
  0.04545454545454545),
 (("Cotton's home",), 0.04545454545454545)]

In [24]:
def compute_accuracy_of_model(model):
    total_number_of_questions = 0
    correct_answers = 0
    wrong_predictions = []

    false_positives = []
    dlist = dev_list[:100]
    for index, text in tqdm(enumerate(dlist), total=len(dlist)):

        all_answers = get_all_answers(dev_dict, index)
        total_questions = len(all_answers)        
        
        for number in range(total_questions):
            small_text = get_text_from_data_item(dev_dict['data'][index], 
                                                 max_num_questions=8,
                                                 question_number=number,
                                                 last_question=False)
            predictions = generate_multiple_answers(model, small_text)
            predictions = group_similar_answers_and_get_scores(predictions)
            all_predictions = []
            #for prediction in predictions[:2]:
            #    all_predictions += prediction[0]
            best_prediction_score = predictions[0][1]
            for prediction in predictions:
                if prediction[1]/best_prediction_score < 0.8:
                    continue
                all_predictions += prediction[0]
            print(all_predictions)
            for prediction in all_predictions:
                it_was_answered = False
                if not prediction:
                    print('NO PREDICTION!!')
                    prediction = 'unknown'
                prediction = prediction.replace('.', '').replace('"', '')
                it_was_answered = False
                for label in all_answers[number]:
                    label = label.replace('.', '').replace('"', '')

                    if prediction.lower() != 'unknown' and label.lower() == 'unknown':
                        false_positives.append(prediction)

                    if prediction.lower() == label.lower():
                        correct_answers += 1
                        it_was_answered = True
                        break
                    elif prediction.lower() in label.lower():
                        correct_answers += 1
                        it_was_answered = True
                        break
                    elif label.lower() in prediction.lower():
                        correct_answers += 1
                        it_was_answered = True
                        break
                    else:
                        wrong_predictions.append({'label': label, 'prediction': prediction})
                        
                if it_was_answered:
                    break

            total_number_of_questions += 1

    return correct_answers/total_number_of_questions, wrong_predictions, false_positives

In [25]:
false_positives = None
wrong_answers = None


checkpoint = torch.load('save_adversarial' + str(6))
model.load_state_dict(checkpoint['model_state_dict'])
accuracy, wrong_answers, false_positives = compute_accuracy_of_model(model.cuda())
print('Epoch:', 6, ' with accuracy:', accuracy)

  0%|          | 0/100 [00:00<?, ?it/s]

['white', 'White']
['in a barn', 'a barn']
['no']
['her mom and 5 other sisters', 'her mommy and 5 other sisters']
['orange']
['no']
['she used her fur to paint herself', 'used it to paint herself', 'she painted herself', 'she used it to paint herself', 'painted herself like them', 'painted herself', 'she used her paint', 'she used her paint to paint herself', 'she painted herself like them', 'used her paint to paint herself']
["her mommy's", 'her mommy']
['laughed', 'they laughed']
['in a bucket of water', 'a bucket of water', 'a bucket']
['licked her face', 'they licked her face']


  1%|          | 1/100 [00:12<20:21, 12.33s/it]

['yes']
['Asta']
['The bottle', 'The bottle.', 'A bottle']
["Asta's friend"]
['yes', 'Yes']
['Yes']
['a note', 'A note']
['Yes']
['Asta']
["they took it to Asta's papa", "They took it to Asta's papa.", 'They took it to the papa', 'they took it to his papa', "took it to Asta's papa", "They took it to Asta's papa"]
['Yes']


  2%|▏         | 2/100 [00:22<18:01, 11.04s/it]

...

['for him to visit', 'to visit']
['late twentieth-century development', 'Late twentieth-century', 'late twentieth-century', 'late twentieth century development', 'late twentieth century', 'Late twentieth-century development']
['Limited Overs Internationals', 'Limited overs International', 'Limited overs internationals', 'Limited overs cricket']
['Limited Overs Internationals', 'Limited Overs', 'Limited overs internationals', 'Limited Overs International', 'Limited Overs Internationals Internationals']
['Twenty20 International matches', 'Twenty20 Internationals', 'Twenty20 International']
['One Day International', 'A One Day International', 'One Day International matches']
['5 January 1971', '1971']
['5 January 1971', '1971']
['Melbourne', 'Melbourne Cricket Ground']
['no', 'No']
['Australia and England']
['Australia']
['England']
['40 wickets', '5 wickets']
['a red ball.', 'a red ball']
['the rival World Series Cricket competition', 'World Series Cricket', 'World Series Cricket competi

 99%|█████████▉| 99/100 [28:32<00:19, 19.25s/it]

['2001.', '2001']
['the long night', 'a long night', 'long night']
['Billy']
['Byrne']
['he was a mucker', 'a mucker', 'mucker']
['no', 'No']
['Anthony Harding']
['Mallory']
['Mallory']
['Mallory']


100%|██████████| 100/100 [28:42<00:00, 17.23s/it]

['he was sure that the old love would be aroused', "he was sure that the old love would be aroused in the girl's breast", 'he was a prospective son-in-law']
Epoch: 6  with accuracy: 0.7346809854706254


In [21]:
accuracy

0.6910928616550853

## save_small 6
### num_replicas=5, len(dlist)=10 
Epoch: 6  with accuracy: 0.8439716312056738

## save_adversarial 6
### num_replicas=5, len(dlist)=100
Epoch: 6  with accuracy: 0.8313329121920404

### num_replicas=7, len(dlist)=100
Epoch: 6  with accuracy: 0.8458622867972204

### num_replicas=10, len(dlist)=100
Epoch: 6  with accuracy: 0.868603916614024

### num_replicas=20, len(dlist)=100
0.8989260897030954

## Using only most frequent answer
### num_replicas=25, len(dlist)=10
0.7092198581560284

## Using the whole group of most frequent answers
### num_replicas=25, len(dlist)=10
0.7659574468085106
### num_replicas=25, len(dlist)=100
0.7195198989260897

## Using the first two groups of most frequent answers
### num_replicas=25, len(dlist)=100
0.8439671509791535

## Using all groups over a threshold
### num_replicas=25, len(dlist)=100, threshold>0.4
0.6910928616550853

### num_replicas=25, len(dlist)=100, score/best_prediction > 0.80.
0.7346809854706254

## TEST WITH 2 groups and fixed YES/NO answers



# TODO
#### USE DROPOUT ON A BATCH OF IDENTICAL INPUTS!
This way there is no need to run the model 5-6-7 times

In [114]:
checkpoint = torch.load('save_adversarial' + str(1))
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [105]:
def subfinder(lst, pattern, offset=0):
    index1 = offset
    index2 = offset
    
    while index1 < len(lst):
        if lst[index1] != pattern[index2]:
            index2 = 0
            index1 += 1
            continue
        index1 += 1
        index2 += 1
        if index2 == len(pattern):
            return index1 - index2
        
    return -1

In [86]:
def generate_answer_with_attention(model, prompt):
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 50
    tokens_length = tokens.shape[1]
    if tokens_length + _length > 1024:
        return ''
    output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length,
             #temperature=0,
             pad_token_id=50256
    )
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    offset = len(prompt)
    start = offset + 1
    end = output.find('\n', start)
    att_output = model(tokens.cuda(), output_attentions=True)
    weights = torch.mean(att_output.attentions[11][0][2][-4:-1], dim=-2)
    return output[start:end].split(':')[-1].strip(), weights, tokens

In [115]:
def generate_multiple_answer_with_attention(model, prompt, num_replicas=10):
    model.train()
    outputs = []
    with torch.no_grad():
        tokens = tokenizer.encode(prompt, return_tensors='pt')
        tokens = tokens.repeat(num_replicas,1)
        _length = 50
        tokens_length = tokens.shape[1]
        if tokens_length + _length > 1024:
            return ''

        
        output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length,
             pad_token_id=50256
        )
        for index in range(num_replicas):
            text = tokenizer.decode(output[index, :], skip_special_tokens=True)
            offset = len(prompt)
            start = offset + 1
            end = text.find('\n', start)
            att_output = model(tokens.cuda(), output_attentions=True)
            weights = torch.mean(att_output.attentions[11][0][2][-4:-1], dim=-2)
            outputs.append((text[start:end].split(':')[-1].strip(), weights, tokens))

    return outputs

## TODO IMPROVEMENTS
* Do not just use layer 11
* Average among all "similar" answers (same group)

In [149]:
small_text = get_text_from_data_item(dev_dict['data'][0], 
                                                 max_num_questions=8,
                                                 question_number=0,
                                                 last_question=False)
tokens = tokenizer.encode(small_text, return_tensors='pt')
answer_weights_tokens_list = generate_multiple_answer_with_attention(model, small_text)

In [150]:
words = [tokenizer.convert_tokens_to_string(
         tokenizer.convert_ids_to_tokens([item], skip_special_tokens=True)[0])
         for item in tokens[0]]

In [161]:
answer_weights_tokens_list[7]

('orange',
 tensor([3.6089e-01, 8.1199e-05, 1.0805e-04, 6.5980e-05, 8.9267e-04, 3.7233e-05,
         0.0000e+00, 1.0273e-04, 8.4571e-05, 6.3877e-05, 1.2993e-04, 7.8752e-04,
         1.6993e-04, 7.4322e-06, 8.6734e-04, 4.2481e-03, 3.9622e-03, 5.9246e-04,
         9.9492e-05, 1.0623e-04, 7.9502e-04, 2.6960e-04, 1.0184e-04, 5.8681e-03,
         9.3598e-04, 4.1873e-05, 5.5301e-04, 4.0890e-04, 1.5313e-03, 1.4488e-03,
         1.1528e-02, 1.7011e-03, 8.0181e-04, 5.3033e-03, 4.9567e-02, 1.8595e-03,
         1.9864e-02, 5.4416e-03, 4.1015e-03, 8.2716e-04, 1.9289e-03, 7.8403e-05,
         1.2960e-04, 1.7088e-04, 1.0379e-03, 3.4712e-03, 2.4533e-04, 4.0942e-04,
         8.7080e-05, 8.9755e-04, 1.3617e-03, 2.6039e-04, 8.5770e-05, 1.7864e-04,
         2.7513e-03, 2.0561e-04, 2.9695e-03, 2.6667e-03, 2.8464e-03, 1.2307e-03,
         2.8963e-03, 1.6620e-03, 9.3932e-04, 1.6716e-03, 1.9829e-04, 9.3560e-04,
         2.3852e-04, 2.8511e-04, 1.1969e-03, 1.4051e-05, 1.1094e-04, 6.2961e-04,
         1.8531e-

In [163]:
from colored import colored, fg, bg, attr


answer, weights, tokens = answer_weights_tokens_list[7]

regular_words = [' ', '"', '\'', '.', ',', '."', ':', '?', '!', '".']

start = subfinder(words, ['\n', '\n', 'Story', ':']) + 4
end = subfinder(words, ['\n', '\n', 'Discussion', ':'])
maximum = torch.max(weights[start:end])
minimum = torch.min(weights[start:end])


for word, weight, in zip(words, weights):
    int_color = int(max(0, (1 - (weight - minimum)/(maximum - minimum))) * 20 + 235)
    if int_color < 240:
        int_color = 0
        
    else:
        int_color = 255
        
    if word in regular_words:
        int_color = 255
        
    if int_color > 245:
        color = fg('black') + bg(int_color)
        
    else:
        color = fg('white') + bg(int_color)
        
    res = attr('reset')
    print(color + word + res, end='')
    
    

In the text below two people are discussing a story.

Story:
Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. 

"What are you doing, Cotton?!" 

"I only wanted to be more like you". 

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pret